In [8]:
# # # SET UP # # #

# import packages
import camelot
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import warnings
from datetime import datetime as dt, date, timedelta as td
import calendar
import numpy as np

# initialize
parser = 'html.parser'

# suppress warnings
pd.options.mode.chained_assignment = None


In [2]:
# # # GET LINKS TO POPULATION DATA SETS FROM IDOC SITE # # #

resp = urllib.request.urlopen('https://www2.illinois.gov/idoc/reportsandstatistics/Pages/Prison-Population-Data-Sets.aspx')
soup = BeautifulSoup(resp, parser, from_encoding=resp.info().get_param('charset'))

link_list = []
for link in soup.find_all('a', href=True):
    if '.xls' in link['href']:
        link_list.append('https://www2.illinois.gov' + link['href'])
# print(link_list)


In [3]:
# # # COMBINE THE POPULATION DATA SETS # # #

# add each doc date to the df - different url formats need to be read

df_list = []
cnt = 0

for link in link_list:
    cnt += 1
    print(str(cnt) + ' of ' + str(len(link_list)) + ' - ' + link)
    
    # read in the spreadsheet
    df = pd.read_excel(link, skiprows=5)
    
    # https://www2.illinois.gov/idoc/reportsandstatistics/Documents/March%202021%20%20Prison%20Stock.xls
    try:
        doc_month_name = link.split(r'Documents/')[1].split(r'%20')[0]
        doc_month_name = doc_month_name[:3]
        doc_month_number = dt.strptime(doc_month_name, "%b").month
        doc_year = int(link.split(r'Documents/')[1].split(r'%20')[1][:4])
        doc_day = calendar.monthrange(doc_year, doc_month_number)[1]
        
    # https://www2.illinois.gov/idoc/reportsandstatistics/Documents/Internet%20Data%20Set%20Prison_Stock_Dec_2017.xls
    except:
        try:
            doc_month_name = link.split('_')[2]
            doc_month_name = doc_month_name[:3]
            doc_month_number = dt.strptime(doc_month_name, "%b").month
            try:
                doc_year = int(link.split('_')[3][:4])
            # _Jun%202017
            except:
                doc_year = int(link.split('_')[2].split('%20')[1][:4])
            doc_day = calendar.monthrange(doc_year, doc_month_number)[1]
        
        # https://www2.illinois.gov/idoc/reportsandstatistics/Documents/Prison%20Population%206-30-16%20Data%20Set.xls
        except:
            doc_month_number = int(link.split('-')[0].split('%20')[2])
            doc_year = int('20' + link.split('-')[2].split('%20')[0])
            doc_day = calendar.monthrange(doc_year, doc_month_number)[1]
            print(date(doc_year, doc_month_number, doc_day))
            
    # only one like this
    if link == 'https://www2.illinois.gov/idoc/reportsandstatistics/Documents/Dec%2031%202019%20Prison%20Stock%20pop.xls':
        doc_month_number = 12
        doc_year = 2019
        doc_day = 31
        
    # create the doc date
    df['Doc Date'] = date(doc_year, doc_month_number, doc_day)

    # add the df to the list
    df_list.append(df)
    
# combine the dfs
df = pd.concat(df_list)

df0 = df.copy()

# no blank Doc Dates at this point
    

1 of 39 - https://www2.illinois.gov/idoc/reportsandstatistics/Documents/June%202021%20Prison%20Stock.xls
2 of 39 - https://www2.illinois.gov/idoc/reportsandstatistics/Documents/March%202021%20%20Prison%20Stock.xls
3 of 39 - https://www2.illinois.gov/idoc/reportsandstatistics/Documents/Dec%202020%20%20Prison%20Stock.xls
4 of 39 - https://www2.illinois.gov/idoc/reportsandstatistics/Documents/Sept%202020%20Prison%20Pop.xls
5 of 39 - https://www2.illinois.gov/idoc/reportsandstatistics/Documents/June%202020%20Prison%20Stock%20Pop.xls
6 of 39 - https://www2.illinois.gov/idoc/reportsandstatistics/Documents/March%202020%20Prison%20Stock.xls
7 of 39 - https://www2.illinois.gov/idoc/reportsandstatistics/Documents/Dec%2031%202019%20Prison%20Stock%20pop.xls
8 of 39 - https://www2.illinois.gov/idoc/reportsandstatistics/Documents/September%202019%20Prison%20Stock.xls
9 of 39 - https://www2.illinois.gov/idoc/reportsandstatistics/Documents/June%202019%20Prison%20Stock.xls
10 of 39 - https://www2.illin

In [4]:
# # # CLEAN DATA/CONSOLIDATE DUPE COLUMNS # # #

# remove notes at end of each doc
df = df[~pd.isnull(df['Name'])]
df.reset_index(inplace=True)
df['Idx'] = df.index

# rename dupe columns so they can be consolidated programmatically
df.rename(columns={'Projected Discharge  Date2': 'Projected Discharge Date1', 'Current Admission Type': 'Admission Type1'}, inplace=True)
   
len1 = len(df)
dupe_col_name_list = ['Projected Mandatory Supervised Release (MSR) Date', 'Projected Discharge Date', 'Current Admission Date', 'Admission Type']
for col_name in dupe_col_name_list:
    
    # get the rows where the main col is blank
    blank_col_df = df[pd.isnull(df[col_name])]
    
    # fill the main col with the duped col vals for each dupe col when the main col is blank
    len2 = len(blank_col_df)
    blank_col_fill_df_list = []
    for dupe_num in range(1, 5):
        if col_name + str(dupe_num) in df.columns:
            blank_col_fill_df = blank_col_df[(~pd.isnull(blank_col_df[col_name + str(dupe_num)]))]
            blank_col_fill_df[col_name] = blank_col_fill_df[col_name + str(dupe_num)]
            blank_col_fill_df_list.append(blank_col_fill_df)
            blank_col_df = blank_col_df[~blank_col_df.index.isin(blank_col_fill_df.index)]
    
    # combine the rows that were able to be filled with those that weren't (where the main and all dupe cols were blank)
    all_blank_col_df = blank_col_df.copy()
    blank_col_df = pd.concat(blank_col_fill_df_list)
    blank_col_df = pd.concat([all_blank_col_df, blank_col_df])
    if len(blank_col_df) != len2:
        print('ERROR: ' + col_name + ' length mismatch')
        1/0
        
    # combine rows where the main col was blank with rows where the main col wasn't blank
    non_blank_col_df = df[~pd.isnull(df[col_name])]
    df = pd.concat([blank_col_df, non_blank_col_df])
    if len(df) != len1:
        print('ERROR: Length mismatch')
        1/0

# drop the dupe columns
for col_name in dupe_col_name_list:
    for dupe_num in range(1, 5):
        if col_name + str(dupe_num) in df.columns:
            df.drop(col_name + str(dupe_num), axis=1, inplace=True)

# remove empty columns
for col in df.columns:
    if df[col].isnull().all():
        df.drop(col, axis=1, inplace=True)
    
print(df.columns)
            
df1 = df.copy()


Index(['index', 'IDOC #', 'Name', 'Date of Birth', 'Sex', 'Race',
       'Veteran Status', 'Current Admission Date', 'Admission Type',
       'Parent Institution', 'Custody Date', 'Sentence Date', 'Crime Class',
       'Holding Offense', 'Sentence Years', 'Sentence Months',
       'Truth in Sentencing', 'Sentencing County', 'Doc Date',
       'Projected Mandatory Supervised Release (MSR) Date',
       'Projected Discharge Date', 'Idx'],
      dtype='object')


In [5]:
# CONVERT ALL DATES TO DATES

# have to make some guesses, e.g. 1211983 = 1/21/1983 or 12/1/1983?

len1 = len(df)


def get_date_parts_for_unknown_mmdd(unknown_part_placement_df):
    correct_placement_df_list = []

    for i in range(1, 3):
        unknown_part_placement_df['month'] = unknown_part_placement_df['mmdd'].apply(lambda x: int(x[:i]))
        unknown_part_placement_df['day'] = unknown_part_placement_df['mmdd'].apply(lambda x: int(x[i:]))
        correct_placement_df = unknown_part_placement_df[(unknown_part_placement_df['month'] > 0) & (unknown_part_placement_df['day'] > 0) 
                                                         & (((unknown_part_placement_df['month'].isin([1, 3, 5, 7, 8, 10, 12])) & (unknown_part_placement_df['day'] <= 31))
                                                            | ((unknown_part_placement_df['month'].isin([4, 6, 9, 11])) & (unknown_part_placement_df['day'] <= 30))
                                                            | ((unknown_part_placement_df['month'] == 2) & 
                                                                ((unknown_part_placement_df['day'] <= 28) | ((unknown_part_placement_df['day'] <= 29) & (unknown_part_placement_df['year'] % 4 == 0)))))]
        correct_placement_df_list.append(correct_placement_df)
        
        # get rid of the correct ones for the next iteration, in case they can work both ways
        unknown_part_placement_df = unknown_part_placement_df[~unknown_part_placement_df.index.isin(correct_placement_df.index)]
        
    full_correct_placement_df = pd.concat(correct_placement_df_list)
    
    return full_correct_placement_df


def convert_to_dates(orig_df, col_name, max_year):
    
    # sort the df
    orig_df.sort_values(by='Idx', inplace=True)

    
    # # # CLEAN NULLS AND DATA TYPES # # #
    
    # remove nulls
    # ----- 32 for dob, 87758 for discharge -----
    na_date_df = orig_df[pd.isnull(orig_df[col_name])]
    print('\nNA times')
    print(len(na_date_df))
    df = orig_df[~pd.isnull(orig_df[col_name])]
    
    # get data types - 0 NaType for dob, custody date, sentence date
    df['type'] = df[col_name].apply(lambda x: str(type(x)))
    print('\ndata types')
    print(df[['type', col_name]].groupby('type').count())

    # make the floats strings
    float_date_df = df[df['type'] == "<class 'float'>"]
    float_date_df[col_name] = float_date_df[col_name].apply(lambda x: str(int(x)))

    # get only string records
    str_date_df = df[df['type'] == "<class 'str'>"]
    str_date_df = pd.concat([str_date_df, float_date_df])

    # get the timestamp records
    ts_date_df = df[~df.index.isin(str_date_df.index)]

    # null out non-dates and add to na_df
    # ----- 68 for dob, 5 for cad, 38 for custody date, 10 for sentence date, 1765 for msr, 4963 for discharge -----
    #   date string length is 1
    print('\nNA - string length')
    str_date_df['len'] = str_date_df[col_name].apply(lambda x: len(x))
    print(str_date_df[['len', col_name]].groupby('len').count())
    null_str_date_df = str_date_df[str_date_df['len'] == 1]
    na_date_df = pd.concat([na_date_df, null_str_date_df])
    
    #   date string has spaces
    str_date_df = str_date_df[str_date_df['len'] > 1]
    print('\nNA - records with spaces')
    str_date_df[col_name] = str_date_df[col_name].str.strip()
    null_str_date_df = str_date_df[str_date_df[col_name].str.contains(' ')]
    print(len(null_str_date_df))
    na_date_df = pd.concat([na_date_df, null_str_date_df])

    str_date_df = str_date_df[~str_date_df.index.isin(null_str_date_df.index)]
    
    len2 = len(str_date_df)

    
    # # # DETERMINE IF YYYY IS AT BEGINNING OR END OF DATE STRINGS # # #

    # create fields assuming yyyy is at beginning or end
    str_date_df['year at beginning'] = str_date_df[col_name].apply(lambda x: int(x[:4]))
    str_date_df['year at end'] = str_date_df[col_name].apply(lambda x: int(x[-4:]))
    
    # BEGINNING
    # yyyy is assumed to be at the beginning if it's between 1920 and the max year passed in
    suspected_year_at_beginnning_df = str_date_df[(str_date_df['year at beginning'] >= 1920) & (str_date_df['year at beginning'] <= max_year)]
    
    # set the mm dd yyyy assuming yyyy at beginning
    suspected_year_at_beginnning_df['mmdd'] = suspected_year_at_beginnning_df[col_name].apply(lambda x: x[4:])
    suspected_year_at_beginnning_df['year'] = suspected_year_at_beginnning_df[col_name].apply(lambda x: float(x[:4]))
    
    # test that the mm dd yyyy all are possible and keep those that make sense
    year_at_beginning_df = get_date_parts_for_unknown_mmdd(suspected_year_at_beginnning_df)
    year_at_beginning_df['year placement'] = 'beginning'
    suspected_year_at_beginnning_df['year'] = suspected_year_at_beginnning_df[col_name].apply(lambda x: x[:4])
    
    # END
    # yyyy is at the end if it's not at the beginning and is between 1920 and the max year passed in
    suspected_year_at_end_df = str_date_df[(~str_date_df.index.isin(year_at_beginning_df.index)) & (str_date_df['year at end'] >= 1920) & (str_date_df['year at end'] <= max_year)]
    
    # set the mm dd yyyy assuming yyyy at end
    suspected_year_at_end_df['mmdd'] = suspected_year_at_end_df[col_name].apply(lambda x: x[:-4])
    suspected_year_at_end_df['year'] = suspected_year_at_end_df[col_name].apply(lambda x: float(x[-4:]))
    
    # test that the mm dd yyyy all are possible and keep those that make sense
    year_at_end_df = get_date_parts_for_unknown_mmdd(suspected_year_at_end_df)
    year_at_end_df['year placement'] = 'end'
    suspected_year_at_end_df['year'] = suspected_year_at_end_df[col_name].apply(lambda x: x[-4:])
    
    # OTHER
    # get the items that could not be coverted to a date because setting the yyyy at the beginning or end didn't make sense
    # ----- 38917, formatted like 99989999 -----
    non_date_df = str_date_df[(~str_date_df.index.isin(year_at_beginning_df.index)) & (~str_date_df.index.isin(year_at_end_df.index))]
    print('\nNA - non-dates')
    print(len(non_date_df))
    print(non_date_df[col_name].iloc[0:10])
    na_date_df = pd.concat([na_date_df, non_date_df])
    len2 = len2 - len(non_date_df)
    
    # keep the values where the date can be formed
    str_date_df = pd.concat([year_at_beginning_df, year_at_end_df])
    if len(str_date_df) != len2:
        print('finding the year placement changed the length')
        1/0
    
    
    # # # CONVERT TO DATES # # #
    
    # pad the date parts where necessary
    for dp in ['year', 'month', 'day']:
        str_date_df[dp] = str_date_df[dp].apply(lambda x: str(int(x)))
        str_date_df['len'] = str_date_df[dp].apply(lambda x: len(x))
        need_to_pad_df = str_date_df[str_date_df['len'] == 1]
        need_to_pad_df[dp] = '0' + need_to_pad_df[dp]
        padded_df = str_date_df[str_date_df['len'] != 1]
        str_date_df = pd.concat([need_to_pad_df, padded_df])

    if len(str_date_df) != len2:
        print('padding dates changed the length')
        1/0

#     print('\nyears')
#     print(str_date_df['year'].unique())
#     print('months')
#     print(str_date_df['month'].unique())
#     print('days')
#     print(str_date_df['day'].unique())
    
    # convert date strings to the same format
    str_date_df[col_name + ' str'] = str_date_df['year'].add('-' + str_date_df['month'])
    str_date_df[col_name + ' str'] = str_date_df[col_name + ' str'].add('-' + str_date_df['day'])
    
    # convert to dates
    str_date_df[col_name] = str_date_df[col_name + ' str'].apply(lambda x: dt.strptime(x, '%Y-%m-%d'))
    
    # drop the date strings
    str_date_df.drop(col_name + ' str', axis=1, inplace=True)
    
    date_df = pd.concat([str_date_df, ts_date_df])
    date_df[col_name] = date_df[col_name].apply(lambda x: x.date())
    
    
    # null out null dates
    na_date_df[col_name] = None
    
    
    # combine string dates with timestamp dates and non-dates
    df = pd.concat([str_date_df, ts_date_df, na_date_df])
    if len(df) != len1:
        print('final changed the length')
        1/0
        
    return df


# convert dates for each of the listed cols
for col_name in ['Date of Birth', 'Current Admission Date', 'Custody Date', 'Sentence Date', 'Projected Mandatory Supervised Release (MSR) Date', 'Projected Discharge Date']:
    
    print('\n\n\nCONVERTING ' + col_name.upper() + '...')
    
    if col_name[:9] == 'Projected':
        max_year = 2300
    else:
        max_year = int(date.today().year)
    
    df = convert_to_dates(df, col_name, max_year)

    
# print('final length compare')
# print(len1)
# print(len(df))
        
df2 = df.copy()
  




CONVERTING DATE OF BIRTH...

NA times
32

data types
                                                    Date of Birth
type                                                             
<class 'float'>                                            511077
<class 'pandas._libs.tslibs.timestamps.Timestamp'>        1101458
<class 'str'>                                               45571

NA - string length
     Date of Birth
len               
1               68
7           382874
8           173706

NA - records with spaces
0

NA - non-dates
0
Series([], Name: Date of Birth, dtype: object)



CONVERTING CURRENT ADMISSION DATE...

NA times
0

data types
                                                    Current Admission Date
type                                                                      
<class 'float'>                                                     511077
<class 'pandas._libs.tslibs.timestamps.Timestamp'>                 1101490
<class 'str'>                             

In [6]:
# # # CREATE ID AND GET LATEST DOC DATE # # #

# id = idoc # + admission date
len1 = len(df)
blank_date_df = df[pd.isnull(df['Current Admission Date'])]
non_blank_date_df = df[~pd.isnull(df['Current Admission Date'])]

non_blank_date_df['Current Admission Date str'] = non_blank_date_df['Current Admission Date'].apply(lambda x: dt.strftime(x, '%Y%m%d'))
non_blank_date_df['ID'] = non_blank_date_df['IDOC #'].add(non_blank_date_df['Current Admission Date str'])
non_blank_date_df.drop('Current Admission Date str', axis=1, inplace=True)

blank_date_df['ID'] = non_blank_date_df['IDOC #']
df = pd.concat([blank_date_df, non_blank_date_df])

if len(df) != len1:
    print(len(df))
    print(len1)
    print('blanking dates changed the length')
    1/0

# get most recent doc date
doc_date_series = df[['ID', 'Doc Date']]
doc_date_series.rename(columns={'Doc Date': 'Last Doc Date'}, inplace=True)
doc_date_series = doc_date_series.groupby('ID')['Last Doc Date'].max()
df = df.merge(doc_date_series, how='inner', on='ID')

# remove unecessary columns
print(df.columns)
df.drop(['Idx', 'type', 'len', 'year at beginning',
       'year at end', 'mmdd', 'year', 'month', 'day', 'year placement'], axis=1, inplace=True)
# (need to drop Doc Date to remove all duplicates)
        
# remove duplicates
df.drop_duplicates(inplace=True)

df3 = df.copy()


Index(['index', 'IDOC #', 'Name', 'Date of Birth', 'Sex', 'Race',
       'Veteran Status', 'Current Admission Date', 'Admission Type',
       'Parent Institution', 'Custody Date', 'Sentence Date', 'Crime Class',
       'Holding Offense', 'Sentence Years', 'Sentence Months',
       'Truth in Sentencing', 'Sentencing County', 'Doc Date',
       'Projected Mandatory Supervised Release (MSR) Date',
       'Projected Discharge Date', 'Idx', 'type', 'len', 'year at beginning',
       'year at end', 'mmdd', 'year', 'month', 'day', 'year placement', 'ID',
       'Last Doc Date'],
      dtype='object')


In [9]:
# # # GET RECIDIVISM BASED ON CUSTODY DATE # # #

# can't groupby non-numeric types when using rank; IDOC # is non-numeric
# is there a better way to do this? almost definitely

len1 = len(df)

# get IDOC #s with more than one admission date
df1 = df[['IDOC #', 'Current Admission Date']].drop_duplicates()
# df1['IDOC #'] = df1['IDOC #'].astype(str)
df1 = df1.groupby('IDOC #').count()
df1.reset_index(inplace=True)
df1 = df1[df1['Current Admission Date'] > 1]

mult_admissions_df = df[df['IDOC #'].isin(df1['IDOC #'])][['IDOC #', 'Current Admission Date']].drop_duplicates()

# filter to each IDOC # with multiple admission dates and rank them to get the sentence number
mult_admissions_list = []
for cur_idoc_no in mult_admissions_df['IDOC #'].unique():
    cur_idoc_no_df = mult_admissions_df[mult_admissions_df['IDOC #'] == cur_idoc_no]
    cur_idoc_no_df['Sentence Number'] = cur_idoc_no_df['Current Admission Date'].rank()
    
    # get the time difference between the admission dates
    cur_idoc_no_df.sort_values('Current Admission Date', inplace=True)
    cur_idoc_no_df['Next Admission Date'] = cur_idoc_no_df['Current Admission Date'].shift(-1)
    cur_idoc_no_df['Admission Date Diff'] = cur_idoc_no_df['Next Admission Date'] - cur_idoc_no_df['Current Admission Date']
    
    mult_admissions_list.append(cur_idoc_no_df)
    
mult_admissions_df = pd.concat(mult_admissions_list)

# add the newly calculated info back to the original df
df = pd.merge(df, mult_admissions_df, on=['IDOC #', 'Current Admission Date'], how='left')
df['Sentence Number'].fillna(1, inplace=True)

if len(df) != len1:
    print('ERROR: getting recidivism changed the length of the df')
    1/0
    
# get instances where the admissions are less than a year from each other
short_admis_date_diff_df = df[df['Admission Date Diff'] < td(days=365)]
short_admis_date_diff_df.to_csv('short_admis_date_diff_df.csv')

# drop the unnecessary columns
df.drop(['Next Admission Date', 'Admission Date Diff'], axis=1, inplace=True)

df3a = df.copy()


In [10]:
# # # SMALL POST-PROCESSING CALCULATIONS # # #

len1 = len(df)

# fill in blanks where relevant
df[['Veteran Status']].fillna('Unknown', inplace=True)

# ----- calculate date diff in tableau for now; dt can only handle dates up to a certain year when subtracting which is causing errors -----
# # calc date difference
# # convert to date from datetime
# new_col_dict = {'Projected Mandatory Supervised Release (MSR) Date': 'Est Tm Until Release (Days)', 'Projected Discharge Date': 'Est Tm Until Discharge (Days)'}
# for col in ['Projected Mandatory Supervised Release (MSR) Date', 'Projected Discharge Date']:
#     print('\n')
#     print(col)
#     no_blank_df = df[~pd.isnull(df[col])]
# # #     no_blank_df[col] = no_blank_df[col].apply(lambda x: x.date())
# #     no_blank_df = no_blank_df[(~pd.isnull(no_blank_df['Doc Date'])) & (no_blank_df[col] < date(9999, 1, 1))]
    
#     col_time_py = no_blank_df[col].apply(lambda x: dt.fromtimestamp(x.timestamp()))
#     doc_date_dt = no_blank_df['Doc Date'].apply(lambda x: dt(x.year, x.month, x.day))
#     a = (pd.to_datetime(col_time_py, errors = 'coerce') - pd.to_datetime(doc_date_dt, errors = 'coerce')).dt.days()

#     no_blank_df[new_col_dict[col]] = (no_blank_df[col] - no_blank_df['Doc Date']).dt.days()
    
#     blank_df = df[~df.index.isin(no_blank_df.index)]
#     df = pd.concat(blank_df, no_blank_df)

#     if len(df) != len1:
#         print('ERROR: calculating ' + new_col_dict[col] + ' changed the df length')
#         1/0

# relabel race
other_race_df = df[~df['Race'].isin(['White', 'Black', 'Hispanic'])]
other_race_df['Race Cat'] = 'Other'
race_df = df[df['Race'].isin(['White', 'Black', 'Hispanic'])]
race_df['Race Cat'] = race_df['Race']
df = pd.concat([other_race_df, race_df])
if len(df) != len1:
    print('ERROR: categorizing race changed the df length')
    1/0
    
# categorize sentencing counties
df['Sentencing County'] = df['Sentencing County'].str.upper()
df['Sentencing County'] = df['Sentencing County'].str.replace('ST CLAIR', 'ST. CLAIR')
df['Sentencing County'] = df['Sentencing County'].str.replace('SAINT CLAIR', 'ST. CLAIR')

len1 = len(df)

cook_df = df[df['Sentencing County'] == 'COOK']
cook_df['Sentencing Region'] = 'COOK'

chi_metro_df = df[df['Sentencing County'].isin(['DUPAGE', 'LAKE', 'KANE', 'MCHENRY', 'WILL'])]
chi_metro_df['Sentencing Region'] = 'CHICAGO METRO'

north_il_df = df[df['Sentencing County'].isin(['BOONE', 'BUREAU', 'CARROLL', 'DEKALB', 'GRUNDY', 
                                               'HENRY', 'JO DAVIESS', 'KENDALL', 'LASALLE', 'LEE', 
                                               'OGLE', 'ROCK ISLAND', 'STEPHENSON', 'WHITESIDE', 
                                               'WINNEBAGO'])]
north_il_df['Sentencing Region'] = 'NORTH IL'

cent_il_df = df[df['Sentencing County'].isin(['ADAMS', 'BROWN', 'CALHOUN', 'CASS', 'CHAMPAIGN', 'CHRISTIAN', 
                                              'COLES', 'CUMBERLAND', 'DEWITT', 'DOUGLAS', 'EDGAR', 'FORD', 
                                              'FULTON', 'GREENE', 'HANCOCK', 'HENDERSON', 'IROQUOIS', 'JERSEY', 
                                              'KANKAKEE', 'KNOX', 'LIVINGSTON', 'LOGAN', 'MACON', 'MACOUPIN', 
                                              'MARSHALL', 'MASON', 'MCDONOUGH', 'MCLEAN', 'MENARD', 'MERCER', 
                                              'MONTGOMERY', 'MORGAN', 'MOULTRIE', 'PEORIA', 'PIATT', 'PIKE', 
                                              'PUTNAM', 'SANGAMON', 'SCHUYLER', 'SCOTT', 'SHELBY', 'STARK', 
                                              'TAZEWELL', 'VERMILION', 'WARREN', 'WOODFORD'])]
cent_il_df['Sentencing Region'] = 'CENTRAL IL'

st_louis_df = df[df['Sentencing County'].isin(['MADISON', 'ST. CLAIR'])]
st_louis_df['Sentencing Region'] = 'ST. LOUIS METRO'

south_il_df = df[df['Sentencing County'].isin(['ALEXANDER', 'BOND', 'CLARK', 'CLAY', 'CLINTON', 'CRAWFORD', 
                                               'EDWARDS', 'EFFINGHAM', 'FAYETTE', 'FRANKLIN', 'GALLATIN', 
                                               'HAMILTON', 'HARDIN', 'JACKSON', 'JASPER', 'JEFFERSON', 
                                               'JOHNSON', 'LAWRENCE', 'MARION', 'MASSAC', 'MONROE', 'PERRY', 
                                               'POPE', 'PULASKI', 'RANDOLPH', 'RICHLAND', 'SALINE', 'UNION', 
                                               'WABASH', 'WASHINGTON', 'WAYNE', 'WHITE', 'WILLIAMSON'])]
south_il_df['Sentencing Region'] = 'SOUTH IL'

county_df = pd.concat([cook_df, chi_metro_df, north_il_df, cent_il_df, st_louis_df, south_il_df])

other_county_df = df[~df['Sentencing County'].isin(county_df['Sentencing County'])]
other_county_df['Sentencing Region'] = 'OUT OF STATE/UNKNOWN'
# print(other_county_df['Sentencing County'].unique())

df = pd.concat([county_df, other_county_df])

if len(df) != len1:
    print('ERROR: Adding sentencing region changed the df length')
    1/0
    
df4 = df.copy()


In [13]:
# # # CLEAN THE TIS COLUMN - PCT AND DESC # # #

# convert to string
df['TIS type'] = df['Truth in Sentencing'].apply(lambda x: str(type(x)))
# print(df[['Truth in Sentencing', 'TIS type']].drop_duplicates())

# lifers and SDP are not included
life_sdp_df = df[df['Sentence Years'].isin(['LIFE', 'SDP'])]
life_sdp_df['TIS Pct'] = None
life_sdp_df['TIS Desc'] = life_sdp_df['Sentence Years']

tis_df = df[(~pd.isnull(df['Truth in Sentencing'])) & (~df['Sentence Years'].isin(['LIFE', 'SDP']))]


# CLEAN STRINGS
tis_str_df = tis_df[tis_df['TIS type'] == "<class 'str'>"]

# split by '%' when possible - pct before, desc after
tis_str_pct_df = tis_str_df[tis_str_df['Truth in Sentencing'].str.contains('%')]
tis_str_pct_df['TIS Pct'] = tis_str_pct_df['Truth in Sentencing'].apply(lambda x: float(x.split('%')[0]) / 100)
tis_str_pct_df['TIS Desc'] = tis_str_pct_df['Truth in Sentencing'].apply(lambda x: x.split('%')[1].strip())

# where no '%' only a description exists - if desc is day-for-day, 50%, o/w, set the pct to null
tis_str_no_pct_df = tis_str_df[~tis_str_df.index.isin(tis_str_pct_df.index)]
dfd_df = tis_str_no_pct_df[tis_str_no_pct_df['Truth in Sentencing'] == 'Day-for-Day']
dfd_df['TIS Pct'] = 0.5
non_dfd_df = tis_str_no_pct_df[tis_str_no_pct_df['Truth in Sentencing'] != 'Day-for-Day']
non_dfd_df['TIS Pct'] = None
tis_str_no_pct_df = pd.concat([dfd_df, non_dfd_df])
tis_str_no_pct_df['TIS Desc'] = tis_str_no_pct_df['Truth in Sentencing']

tis_str_df = pd.concat([tis_str_pct_df, tis_str_no_pct_df])


# CLEAN NUMBERS
tis_num_df = tis_df[tis_df['TIS type'] != "<class 'str'>"]

# only pct exists - set desc to null
tis_num_df['TIS Pct'] = tis_num_df['Truth in Sentencing'].astype(float)
tis_num_df['TIS Desc'] = None

tis_df = pd.concat([tis_num_df, tis_str_df])

# SET OTHERS TO NULL
non_tis_df = df[~df.index.isin(tis_df.index)]
tis_num_df['TIS Pct'] = None
tis_num_df['TIS Desc'] = None

df = pd.concat([tis_df, non_tis_df])

if len(df) != len1:
    print('ERROR: cleaning TIS changed the df length')
    1/0
    
df5 = df.copy()

In [14]:
# # # ADD HOLDING OFFENSE CATEGORIES # # #

len1 = len(df)

df['Holding Offense'] = df['Holding Offense'].apply(lambda x: str(x).strip())

injury_death_df = df[df['Holding Offense'].isin(['AB/NEGLECT ELD/DEATH/CAREGIVER', 'AGG BATTERY CHILD <13/GREAT BOD HARM', 'AGG BATTERY OF A CHILD', 'AGG BATTERY W/FIREARM/PERSON', 'AGG BATTERY/CHILD <13/PERM DISABL', 'AGG BATTERY/DISCHARGE FIREARM', 'AGG BATTERY/FIREARM/OFF/FIREMAN', 'AGG BATTERY/GREAT BOD HARM/60+', 'AGG BATTERY/GREAT BOD HARM/FLAME SUBST', 'AGG BATTERY/GREAT BOD HARM/PC OFF', 'AGG BATTERY/GREAT BODILY HARM', 'AGG BATTERY/HARM/PEACE OFFICER', 'AGG BATTERY/MERCHANT', 'AGG BTRY/UNLAWFUL DEL/CON SUB', 'AGG KIDNAPING/INFLICT HARM', 'AGG STALKING/BODILY HARM', 'AGG STALKING/BODILY HARM/2ND', 'AGG VEH HIJACKING/DISCH/HARM', 'AGGRAVATED BATTERY W/FIREARM', 'ARMED ROBBERY/DISCHARGE/HARM', 'DOM BTRY/BOD HARM/4+ PRI CONV', 'DOMESTIC BATTERY/BODILY HARM', 'HEINOUS BATTERY', 'HOME INVASION/CAUSE INJURY', 'HOME INVASION/DISCH FIREARM/HARM', 'INTIMIDATION/PHYSICAL HARM', 'MURDER/2ND DEGREE MURDER', 'MURDER/2ND DEGREE/PROVOCATION', 'MURDER/2ND DEGREE/UNREASON', 'MURDER/HOMICIDE/UNBORN CHILD', 'MURDER/INTENT TO KILL/INJURE', 'MURDER/OTHER FORCIBLE FELONY', 'MURDER/STRONG PROB KILL/INJURE', 'RECKLESS CONDUCT/GREAT BODILY HARM', 'VOLUN MANSL/NEGL-DEATH ANOTHER', 'DRUG INDUCED HOMICIDE', 'AGG DUI BODILY HARM/.10+/CHILD', 'AGG DUI/ACCIDENT/BODILY HARM', 'AGG DUI/ACCIDENT/DEATH', 'AGG DUI/BODILY HARM W/CHILD', 'AGG DUI/DEATH OF ANOTHER', 'AGG DUI/GREAT BODILY HARM', 'INVOL MANSLAUGHTER ATV/BOAT', 'INVOL MANSLAUGHTER FAMILY/BOAT', 'RECKLESS HOMIC/INCLINE/DEATH 2+', 'RECKLESS HOMIC/TRAFFIC CONTROL', 'RECKLESS HOMICIDE', 'RECKLESS HOMICIDE/.08 ALC/DRUG', 'RECKLESS HOMICIDE/ALC/CONVIC', 'RECKLESS HOMICIDE/ALC/DRUG', 'RECKLESS HOMICIDE/KILL 2 INDIV', 'SNOWMB OUI OF ALCOHOL/DEATH', 'INVOL MANSLAUGHTER/FAMILY MEMBER', 'INVOLUNTARY MANSLAUGHTER'])]
injury_death_df['Holding Offense Cat'] = 'Person - Death/Physical Injury'

injury_death_unclear_df = df[df['Holding Offense'].isin(['AGG ASLT PEACE OFF/FIRE/ER WRK', 'AGG ASLT/OP MOTOR VEH/STRUCK', 'AGG ASLT/STATE IL EMP/WEAPON', 'AGG ASSAULT/CORRECTIONAL EMP', 'AGG ASSAULT/DHS EMPLOYEE', 'AGG ASSAULT/DISCH FIREARM MV', 'AGG ASSAULT/OP MOTOR VEH/PC OFF', 'AGG ASSAULT/USE FOREARM/PEACE OFF', 'AGG BATTERY SR CITIZEN >60 YRS', 'AGG BATTERY/CONTROLLED SUB', 'AGG BATTERY/GOVERNMENT EMP', 'AGG BATTERY/INGEST TOXIC SUBST', 'AGG BATTERY/JUDGE/EMT', 'AGG BATTERY/PEACE OFF/FIREMAN', 'AGG BATTERY/PEACE OFFICER', 'AGG BATTERY/PREGNANT/HANDICAPPED', 'AGG BATTERY/PUBLIC PLACE', 'AGG BATTERY/STRANGLE/PREV CONV', 'AGG BATTERY/TRANSIT EMPLOYEE', 'AGG BATTERY/USE DEADLY WEAPON', 'AGG BATTERY/WEAPON/NO FIREARM', 'AGG DISCH FIREARM/PC OFF/FIREMAN', 'AGG DISCH FIREARM/VEH/PC OFF/FRMAN', 'AGG DISCH SILENCER PERSON/VEH', 'AGG DISCHARGE FIREARM/BLDG', 'AGG DISCHARGE FIREARM/OCC BLDG', 'AGG DISCHARGE FIREARM/OCC VEH', 'AGG DOMESTIC BATTERY/STRANGLE', 'AGG INTIMIDATION PEACE OFFICER', 'AGG KIDNAPING ARMED W FIREARM', 'AGG KIDNAPING/CHILD<13/RETARDED', 'AGG KIDNAPING/CONCEAL IDENT', 'AGG ROBBERY/CONTROLLED SUB', 'AGG ROBBERY/INDICATE ARM W/FIR', 'AGG STALKING/VIO REST ORDER', 'AGG VEHICULAR HIJACKING/DISCHARGE', 'AGG VEHICULAR HIJACKING/FIREARM', 'AGG VEHICULAR HIJACKING/HANDICAPPED', 'AGG VEHICULAR HIJACKING/PASS <16 YRS', 'AGG VEHICULAR HIJACKING/WEAPON', 'AGGRAVATED ASSAULT', 'AGGRAVATED BATTERY/NURSE', 'AGGRAVATED BATTERY/STRANGLE', 'AGGRAVATED BATTERY/VICTIM 60+', 'AGGRAVATED DISCHARGE/FIREARM', 'AGGRAVATED DOMESTIC BATTERY', 'AGGRAVATED KIDNAPING/ARMED', 'AGGRAVATED KIDNAPING/RANSOM', 'AGGRAVATED ROBBERY', 'AGGRAVATED TRAFFICKING/BENEFIT', 'AGGRAVATED UNLAWFUL RESTRAINT', 'AGGRAVATED VEHICULAR HIJACKING', 'ARMED ROBBERY', 'ARMED ROBBERY/ARMED W/FIREARM', 'ARMED ROBBERY/DISCH FIREARM', 'ARMED ROBBERY/NO FIREARM', 'ARMED VIOL/CATEGORY I WEAPON', 'ARMED VIOL/CATEGORY II WEAP/1ST', 'ARMED VIOL/CATEGORY II WEAPON', 'ARMED VIOL/CATEGORY III WEAPON', 'CAUSE CHILD/ENDANGERED/2+', 'CHILD ABDUCTN/CONCEAL/DETAIN', 'COMMUNICATE/DETAIN WITNESS', 'CRIM TRES TO RES/PERS PRESENT', 'DISMEMBERING A HUMAN BODY', 'DOM BTRY/CONTACT/3 PRIOR CONV', 'DOMESTIC BATTERY/ CONTACT/PRIOR', 'DOMESTIC BATTERY/PHY CONTACT/2ND+', 'DOMESTIC BATTERY/VIOL PROTECT ORDER', 'GIVE FALSE BOMB/GAS ALARM', 'HABITUAL CRIMINAL/2 PRIOR CL X', 'HARASS JURORS/WITNESSES', 'HARASS WITNESS', 'HARASSMENT BY PHONE/4TH +', 'HOME INVASION', 'HOME INVASION/ARMED W/FIREARM', 'HOME INVASION/ARMED/FORCE', 'HOME INVASION/DANDEROUS WEAP', 'HOME INVASION/DISCH FIREARM', 'HOME INVASION/FIREARM', 'HOME INVASION/SEX OFFENSE', 'HOME INVASION/VIO AGAINST PERS', 'HRSMT/PREVENT/SERV/SAME VIC', 'HRSMT/THREATEN PERSON/KILL', 'HUMAN TRAFFICKING FOR LABOR/BENEFITS', 'INTIMIDATION/CONTEMPT/RIDICULE', 'INTIMIDATION/CRIMINAL OFFENSE', 'INVOL SERVITUDE/THREAT PHYS HARM', 'KIDNAPING ARMED WITH FIREARM', 'KIDNAPING DISCHARGE FIREARM', 'KIDNAPING W FORCE OR THREAT', 'KIDNAPING/SECRETLY CONFINE', 'PHONE HRSMT/LEWD/THREAT KILL', 'RECKLESS DISCH FIREARM/ENDANGERS', 'ROBBERY', 'ROBBERY/SCHOOL/PLACE WORSHIP', 'ROBBERY/VIC HANDICAP / 60+', 'ROBBERY/VIC HANDICAP OR 60+ YR', 'STALKING BY TRANSMITTING THREAT', 'STALKING/PERSON/SURVEILLANCE', 'STALKING/TRANSMITS THREAT/2ND', 'UNLAWFUL RESTRAINT', 'UNLAWFUL VEHICULAR INVASION', 'VEHICULAR HIJACKING', 'VIO ORDER/PRIOR VIO OF ORDER', 'VIOLATE ORDER/PRIOR DOM BATTERY', 'AB/NEGLECT ELDERLY/CAREGIVER', 'ARMED VIOL/CATEGORY II WEAP/2ND+', 'CONTEMPT', 'KIDNAPING/DECEIT OR ENTICE', 'VIOLATE STALKING NO CONTACT/2+', 'CHILD ABDUCT/LURE/VIC<17/PRIOR', 'WINDOW PEEPING 3RD+', 'ATT AGG BATTERY/CHILD <13/PERM DISABL', 'ATT AGG BATTERY/PEACE OFFICER', 'ATT AGG DISCHARGE FIREARM', 'ATT AGG DISCHARGE FIREARM AT VEH', 'ATT AGG DISCHARGE FIREARM OFFICER', 'ATT AGG VEHICLE HIJACK W WEAPON', 'ATT ARMED ROBBERY/ARMED', 'ATT ARMED ROBBERY/NO FIREARM', 'ATT DRUG-INDUCED HOMICIDE', 'ATT HOME INVASION/ARMED W/FIREARM', 'ATT HOME INVASION/FIREARM', 'ATTEMPT AGG BATTERY OF A CHILD', 'ATTEMPT AGG BATTERY/DISCHARGE FIREARM', 'ATTEMPT AGG DISCHARGE/FIREARM', 'ATTEMPT AGG KIDNAPG/CHILD<13/RETARD', 'ATTEMPT AGG KIDNAPING/INFLICT HARM', 'ATTEMPT AGG ROBBERY/INDICATE ARM W/FIR', 'ATTEMPT AGG VEHICULAR HIJACKING', 'ATTEMPT AGGRAVATED ROBBERY', 'ATTEMPT ARMED ROBBERY', 'ATTEMPT ARMED VIO/CATEGORY I WEAPON', 'ATTEMPT DISARM PC OFF/CORR EMP', 'ATTEMPT HARASS JURORS/WITNESSES', 'ATTEMPT HOME INVASION/CAUSE INJURY', 'ATTEMPT INTENT/HOMICIDE/UNBORN CHILD', 'ATTEMPT KIDNAP ARMED W FIREARM', 'ATTEMPT MURDER/INTENT TO KILL/INJURE', 'ATTEMPT MURDER/OTHER FORCIBLE FELONY', 'ATTEMPT MURDER/STRONG PROB KILL/INJURE', 'ATTEMPT ROBBERY', 'ATTEMPT ROBBERY/VIC HANDICAP OR 60+ YR', 'ATTEMPT SOLICITATION/MURDER/FOR HIRE', 'ATTEMPT VEHICULAR HIJACKING', 'CONSP AGGRAVATED ROBBERY', 'CONSP ARMED ROBBERY', 'CONSP MURDER/INTENT TO KILL/INJURE', 'SOLICIT MURDER/INTENT TO KILL/INJURE', 'SOLICITATION ROBBERY', 'SOLICITATION/MURDER', 'SOLICITATION/MURDER/FOR HIRE', 'CONSP ROBBERY', 'ATTEMPT AGG KIDNAP/CONCEAL IDENTITY', 'ATTEMPT KIDNAPING/SECRETLY CONFINE', 'CONSP AGG BATTERY/DISCHARGE FIREARM'])]
injury_death_unclear_df['Holding Offense Cat'] = 'Person - Death/Physical Injury Unclear'

sex_df = df[df['Holding Offense'].isin(['AGG CHILD PORN/DISSEM FILM', 'AGG CHILD PORN/EXHIBIT', 'AGG CHILD PORN/LEWD EXHIBITION', 'AGG CHILD PORN/PERSON/ANIMAL', 'AGG CHILD PORN/POSS FILM/PHOTO', 'AGG CHILD PORN/POSS FILM/PREV', 'AGG CRIM SEX AB/VIC <18/FAMILY', 'AGG CRIM SEX ABUSE', 'AGG CRIM SEX ABUSE/<5 YR VIC', 'AGG CRIM SEX ABUSE/BODILY HARM', 'AGG CRIM SEX ABUSE/DURING FELONY', 'AGG CRIM SEX ABUSE/FAMILY', 'AGG CRIM SEX ABUSE/FORCE/VIC 9-13', 'AGG CRIM SEX ABUSE/INTELLIG DISABL', 'AGG CRIM SEX ABUSE/VIC <13', 'AGG CRIM SEX ABUSE/VIC 13-16', 'AGG CRIM SEX ABUSE/VIC 13-17', 'AGG CRIM SEX ABUSE/VICTIM <13', 'AGG CRIM SEX ABUSE/VICTIM <9', 'AGG CRIM SEX ASLT/INTEL DISABL', 'AGG CRIM SEX ASLT/THREAT LIFE', 'AGG CRIM SEX ASSAULT/BODILY HARM', 'AGG CRIM SEX ASSAULT/FELONY', 'AGG CRIM SEX ASSAULT/FIREARM', 'AGG CRIM SEX ASSAULT/FORCE VIC9-13', 'AGG CRIM SEX ASSAULT/HANDICAPPED', 'AGG CRIM SEX ASSAULT/RETARDED', 'AGG CRIM SEX ASSAULT/THREAT LIFE', 'AGG CRIM SEX ASSAULT/VIC 60+', 'AGG CRIM SEX ASSAULT/VICTIM <13', 'AGG CRIM SEX ASSAULT/VICTIM <9', 'AGG CRIM SEX ASSAULT/VICTIM >60', 'AGG CRIM SEX ASSAULT/WEAPON', 'AGG CRIM SEXUAL ABUSE/FELONY', 'AGG CRIM SEXUAL ASSAULT/CON SUB', 'AGG CRIM SX AB/VIC 13<18/TRUST', 'AGG CRIM SX ASLT/FORCE VIC9-13', 'AGG IND LIB/CHILD/INFLICT HARM', 'CHILD PORN/FILM/TAPE/PHOTO/ACT', 'CHILD PORN/MOVING DEPICTION', 'CHILD PORN/PERS/ANIM/MOV DPTN', 'CHILD PORN/PERSON/ANIMAL', 'CHILD PORN/POSE/EXHIBITION', 'CHILD PORN/POSS FILM/PHOTOS', 'CHILD PORN/POSS PHOTO/VIC <13', 'CHILD PORN/POSS/MOVING DPTN', 'CHILD PORN/REPRODUCE/MOV DPTN', 'CHILD PORN/REPRODUCE/SELL', 'CHILD PORN/SOL CHILD/MOV DPTN', 'CHILD PORN/SOL/CHILD/APPEAR', 'CHILD PORNOGRAPHY/VICTIM <13', 'CRIM SEX ABUSE/CANT CONSENT/2ND', 'CRIM SEX ASLT/FAM MBR <18/2+', 'CRIM SEX ASLT/FAMILY MEMBER<18', 'CRIM SEX ASLT/FORCE/PREV CONV', 'CRIM SEX ASSAULT/CANT CONSENT', 'CRIM SEX ASSAULT/CANT CONSENT/2ND', 'CRIM SEX ASSAULT/FAMILIES', 'CRIM SEX ASSAULT/FAMILIES/2ND+', 'CRIM SEX ASSAULT/FORCE', 'CRIM SEX ASSAULT/FORCE/2ND+', 'CRIM SEX ASSAULT/SUPERVN VIC 13-17', 'CRIM SEX ASSAULT/VICTIM 13-17', 'CRIM SEX ASSAULT/VICTIM 13-17/2ND', 'CRIM SEXUAL ABUSE/CONSENT', 'CRIM SEXUAL ABUSE/CONSENT/2+', 'CRIMINAL SEX ASSAULT/CONSENT', 'CRIMINAL SEXUAL ABUSE/FORCE', 'DEVIATE SEXUAL ASSAULT', 'GROOMING', 'INDECENT SOL ADULT/<13/PENETRATION', 'INDECENT SOL/AGG CRIM SEX ABUSE', 'INDECENT SOL/CRIM SEX ASSAULT', 'INDECENT SOL/PREDITORY/AGG SEX', 'NONCONSENSUAL  DISSM SEX IMAGE', 'PERMIT SEXUAL ABUSE OF CHILD', 'PRED CRIM SEX ASSLAUT/VICTIM <13', 'PREDATORY CRIM SEX ASSAULT/FIREARM', 'PREDATORY CRIM SEX ASSAULT/HARM', 'PREDATORY CRIMINAL SEXUAL ASSAULT', 'PROM JUV PROST/MINOR <13/RETAR', 'PUBLIC INDECENCY/EXPOSURE/3+', 'RAPE', 'RAPE PRIOR TO 2/1/78', 'SEX RELATIONS WITHIN FAMILIES', 'SEXUALLY DANGEROUS PERSON', 'TRAFFIC SEX SERV MINOR < 17', 'TRAVELING TO MEET A MINOR', 'UNLAWFUL VIDEO/VIC<18/SEX OFF', 'EXPLOIT CHILD/SEX ACT/2ND', 'CUSTODIAL SEXUAL MISCONDUCT', 'INDECENT SOLICIT CHILD/INTERNET', 'ATT AGG CRIM SEX ASSAULT VIC 60+', 'ATT CRIM SEX ASSAULT/FORCE', 'ATT PERMIT SEXUAL ABUSE OF CHILD', 'ATTEMPT AGG CRIM SEX ASLT/BODILY HARM', 'ATTEMPT AGG CRIM SEX ASSAULT', 'ATTEMPT AGG CRIM SEX ASSAULT/BODILY HARM', 'ATTEMPT AGG CRIM SEX ASSAULT/FELONY', 'ATTEMPT AGG CRIM SEX ASSAULT/VICTIM >60', 'ATTEMPT AGG CRIM SEX ASSAULT/WEAPON', 'ATTEMPT AGG CRIM SX AB/>5 YR OLDER VIC', 'ATTEMPT CRIM SEX ASSAULT/FORCE', 'ATTEMPT CRIM SEX ASSAULT/VICTIM 13-17', 'ATTEMPT PRED CRIM SEX ASLT/VICTIM <13', 'ATTEMPT PRED CRIMINAL SEXUAL ASSAULT', 'SOL AGG CRIM SEX ASSAULT/VIC 60+'])]
sex_df['Holding Offense Cat'] = 'Sex'

drugs_df = df[df['Holding Offense'].isin(['15<200 OBJECT/PARTS LSD/ANALOG', 'AGG DEL METH PROTECTED <5 GRAMS', 'AGG DEL METH/PROTECTED/100+ GR', 'AGG DEL METH/PROTECTED/5<15 GRAMS', 'AGG METH MANU/ORGANIZE/100<400 GR', 'AGG METH MANU/PROTECTED<15', 'AGG METH MANU/WORSHIP/<15 GR', 'AGG METH MANUF/APT/<15 GRAMS', 'AGG METH MANUF/APT/100<400 GR', 'AGG METH MANUF/CHILD 15<100 GRAMS', 'AGG METH MANUF/CHILD/<15 GRAMS', 'AGG METH MANUF/CHLD/100<400 GRAMS', 'AGG METH MANUF/CHLD/400+ GR', 'AGG METH MANUF/ORGANIZE/<15 GR', 'CALCULATED CRIM DRUG CONSP', 'CALCULATED CRIM DRUG CONSPIR', 'CANNABIS TRAFFICKING', 'CONT SUBS ACT-MANU/DEL', 'CONTROLLED SUB TRAFFICKING', 'CRIM DRUG CONSPIRACY', 'CRIMINAL DRUG CONSPIRACY', 'CRIMINAL DRUG CONSPIRACY >100 GR', 'DEL CONT SUB<18/PARK/SCHOOL/PUB HOUS', 'DEL CONT/COUNT SUB TO <18', 'DEL METH <5 GRAMS', 'DEL/NON NARC SCHED I/II/SC/HS/PARK', 'DISPOSE METH MANUF WASTE', 'MAN/DEL OTHER AMOUNT AMPHETAMINE', 'MANU 5>15 GRAMS ECSTASY', 'MANU/DEL 01-15 GRAMS COCAINE', 'MANU/DEL 1-14 GRAMS HEROIN', 'MANU/DEL 1<15 GRAM FENTANYL', 'MANU/DEL 10-15 GRAMS HEROIN', 'MANU/DEL 10<15 OBJECTS/PARTS LSD', 'MANU/DEL 10>15 PILLS ECSTASY', 'MANU/DEL 100<400 GR FENTANYL', 'MANU/DEL 100<400 GRAMS COCAINE', 'MANU/DEL 100<400 GRAMS HEROIN', 'MANU/DEL 100<400 GRAMS LSD', 'MANU/DEL 100>400 GR ECSTASY', 'MANU/DEL 15/+ GRAMS HEROIN', 'MANU/DEL 15/+GRAMS COCAINE', 'MANU/DEL 15<100 GRAMS COCAINE', 'MANU/DEL 15<100 GRAMS HEROIN', 'MANU/DEL 15<100 GRAMS LSD', 'MANU/DEL 15>100 GR ECSTASY', 'MANU/DEL 15>200 PILLS ECSTASY', 'MANU/DEL 1500+ PILLS ECSTASY', 'MANU/DEL 200+ SCHED I&II', 'MANU/DEL 200+GRAMS METH/AMPH', 'MANU/DEL 200>600 PILLS ECSTASY', 'MANU/DEL 2ND OR SUBQ OFFENSE IN ACT', 'MANU/DEL 3-15 GRAMS LSD', 'MANU/DEL 400<900 GRAMS COCAINE', 'MANU/DEL 400<900 GRAMS METH', 'MANU/DEL 50-200 GRAMS SCHED I&II', 'MANU/DEL 600<1500 OBJECTS/PARTS LSD', 'MANU/DEL 900+ GRAMS COCAINE', 'MANU/DEL 900+ GRAMS ECSTASY', 'MANU/DEL 900+ GRAMS HEROIN', 'MANU/DEL AMT NARC SCHED I/II/SCHOOL/HS/PARK', 'MANU/DEL BARB ACID/SC/PUB HOUS/PARK', 'MANU/DEL CANNABIS/>5,000 GRAMS', 'MANU/DEL CANNABIS/>500 GRAMS', 'MANU/DEL CANNABIS/10-30 GRAMS', 'MANU/DEL CANNABIS/2000<5000 GRAMS', 'MANU/DEL CANNABIS/30-500 GRAMS', 'MANU/DEL COCAINE/SCHOOL/PUB HOUS/PARK', 'MANU/DEL CONT SUBS (PRIOR 1/1/90)', 'MANU/DEL CONT SUBS/ENHANCED', 'MANU/DEL HEROIN/SCHOOL/PUB HOUS/PARK', 'MANU/DEL OTHER AMOUNT SCHEDULE III', 'MANU/DEL OTHER AMOUNT SCHEDULE IV', 'MANU/DEL OTHER AMT METH', 'MANU/DEL OTHER AMT NARC SCHED I&II', 'MANU/DEL OTHER AMT NARCOTIC SCHED I&II', 'MANU/DEL OTHER NON-NARCOTIC SCHED I&II', 'MANU/DEL SCHED I/II/SCH/HS/PARK', 'MANU/DISTRIB LOOK-ALIKE SUB', 'METH CONSP CONT SUBS >100 GR', 'METH DELIVERY/100<400 GR', 'METH DELIVERY/15<100 GRAMS', 'METH DELIVERY/400<900 GR', 'METH DELIVERY/5<15 GRAMS', 'METH DELIVERY/900+ GR', 'METH DELIVERY<5 GRAMS', 'METH MANUFACTURE/100<400 GR', 'METH MANUFACTURE<15 GRAMS', 'METH MANUFACTURING/100<400 GRAMS', 'METH MANUFACTURING/15<100 GR', 'METH MANUFACTURING/15<100 GRAMS', 'METH MANUFACTURING/400<900 GR', 'METH MANUFACTURING/400<900 GRAMS', 'METH MANUFACTURING/900+ GR', 'METH MANUFACTURING<15 GRAMS', 'METH PRECURSOR TRAFFICKING', 'METH PRECURSOR/15<30 GRAMS', 'METH PRECURSOR/500+ GRAMS', 'METH PRECURSOR<15 GRAMS', 'METHAMPHETAMINE CONSPIRACY', 'METHAMPHETAMINE TRAFFICKING', 'MFG/DEL 100<400 GR HERO/ANLG', 'MFG/DEL 15<100 GR FENTANYL', 'MFG/DEL 15<100 GR HEROIN/ANLG', 'MFG/DEL 400<900 GR HERO/ANLG', 'NARCOTICS RACKETEERING', 'OBTAIN SUBSTANCE BY FRAUD/1ST', 'POSS AMT CON SUB EXCEPT(A)/(D)', 'POSS CANNABIS/>5,000 GRAMS', 'POSS CANNABIS/10-30 GRAM/SUBQ', 'POSS CANNABIS/2,000<5,000 GRAMS', 'POSS CANNABIS/500<2,000 GRAMS', 'POSS CONT SUBS', 'POSS HYPO/SYRINGE/NEEDLES/2ND+', 'POSS METH MANUF MATERIAL', 'POSS OF METH < 5 GRAMS', 'POSS OF METH/ 15<100 GRAMS', 'POSSESS 100<400 GRAMS COCAINE', 'POSSESS 100<400 GRAMS HEROIN', 'POSSESS 15+ GRAMS COCAINE', 'POSSESS 15+ GRAMS HEROIN', 'POSSESS 15+ GRAMS MORPHINE', 'POSSESS 15<100 GRAMS COCAINE', 'POSSESS 15<100 GRAMS HEROIN', 'POSSESS 15<100 GRAMS METH', 'POSSESS 15<200 OBJECT/PART LSD', 'POSSESS 15>200 PILLS ECSTASY', 'POSSESS 1500+ PILLS ECSTASY/ANALOG', 'POSSESS 200+ GRAMS OTHER SCHED I&II', 'POSSESS 400<900 GRAMS COCAINE', 'POSSESS 400<900 GRAMS HEROIN', 'POSSESS 900 + GRAMS COCAINE', 'POSSESS 900 + GRAMS HEROIN', 'POSSESSION OF METH/100<400 GRAMS', 'POSSESSION OF METH/15<100 GRAMS', 'POSSESSION OF METH/400<900 GR', 'POSSESSION OF METH/5<15 GRA', 'POSSESSION OF METH/900+ GR', 'POSSESSION OF METH< 5 GRAMS', 'PRODUCE >200 CANNABIS PLANTS', 'TAMPER W/ANHYD AMM EQUIPMENT', 'USE OF DANGEROUS PLACE/CON SUB', 'USE VEH/STRUCTURE/PROP/METH', 'POSS BLANK COUNTERFEIT SCRIPT', 'POSS CANNABIS/30-500 GRAM/1ST', 'MANU/DEL CONT SUBS', 'MANU/DEL 10<30 GRAMS KETAMINE', 'POSS 400>900 GR ECSTASY/ANALOG', 'PRODUCE 5-20 CANNABIS PLANTS', 'ATT MANU 200>600 PILLS ECSTASY', 'ATT METH PRECURSOR/30<150 GR', 'ATTEMPT MANU/DEL 15/+GRAMS COCAINE', 'ATTEMPT MANU/DEL CANNABIS/>500 GRAMS', 'ATTEMPT MANU/DEL OTHER AMT NARC', 'ATTEMPT POSS METH MANUF MATERIAL', 'ATTEMPT POSSESS 15<100 GRAMS COCAINE', 'CONSP MANU/DEL 01-15 GRAMS COCAINE', 'CONSP MANU/DEL OTHER AMOUNT METH', 'CONSP MFG/DEL CANNABIS/>5,000 GRAMS', 'SOLICIT MANU/DEL 01-15 GRAMS COCAINE', 'ATTEMPT MANU/DEL 01-15 GRAMS COCAINE', 'CONSPIRACY METH MANUFACTURE<15 GRAMS', 'ATTEMPT POSS AMT CON SUB', 'CONSP MANU/DEL 15/+GRAMS COCAINE'])]
drugs_df['Holding Offense Cat'] = 'Drugs'

property_df = df[df['Holding Offense'].isin(['AGG ARSON', 'AGG ARSON/INJURE FIRE/POLICE', 'AGG ARSON/KNOW PEOPLE PRESENT', 'AGG HOME FRAUD VIC >60 >%500', 'AGG ID THEFT/<$300/DISABLED', 'AGG ID THEFT/$10,000-$100,000/60 YRS+', 'AGG ID THEFT/$300-10,000/60 YRS +', 'AGG ID THEFT/AGE 60+/2+', 'AGG ID THFT/10K-100K/GANG ACTS', 'AGG ID THFT/ID STOLEN/VIC 60+', 'AGG INSURANCE FRAUD', 'AGGRAVATED ARSON', 'AGGRAVATED ARSON/BODILY HARM', 'AID/ABET/POSS/SELL STOLEN VEH', 'ARSON', 'ARSON/REAL/PERSONAL PROP>$1', 'BAD CHK/OBTAIN CON PROP/>150', 'BURGLARY', 'BURGLARY W/O CAUSING DAMAGE', 'BURGLARY/SCHOOL/PLACE WORSHIP', 'CAUSING A CATASTROPHE', 'CONTINUING FINANCE CRIME ENTRPRS', 'COUNTERFEIT CREDIT/DEBIT CARD', 'CRIM DMG TO PROP $300-10000', 'CRIMINAL DAMAGE $10000-$100000', 'DEFRAUD FINANC INST/$500-$10,000', 'DEFRAUD FINANCIAL INST/>$100K', 'FIN EXPLOIT ELDERLY/70+/$15K+', 'FINANCE EXPLOIT ELD/DISABL/$300-$5,000', 'FINANCIAL INSTITUTION ROBBERY', 'FORGERY /MAKE OR ALTER DOCUMENT', 'FORGERY/ISSUE DOCUMENT/1 UPC', 'FORGERY/ISSUE/DELIVER DOCUMENT', 'FORGERY/POSSESS W INTENT', 'ID THEFT/OBTAIN/3+ INDIV', 'ID THFT/OBTAIN INFO/COMMIT FEL', 'IDENTITY THEFT >$100,000', 'IDENTITY THEFT/<$300', 'IDENTITY THEFT/$10K - $100K', 'IDENTITY THEFT/$2000-$10,000', 'IDENTITY THEFT/$300-$2000', 'IDENTITY THEFT/KNOWS ID STOLEN', 'KNOW ID STOLEN/MILITARY VIC/2+', 'MOB ACTION', 'ORGANIZE/AGG VEH THEFT CONSPIR', 'ORGANIZER/FIN CRIME ENTRPRS', 'POSS STOLEN VEHICLE > $25,000', 'POSSESS BURGLARY TOOLS', 'RECEIVE/POSS/SELL STOLEN VEH', 'RECEIVE/POSS/SELL STOLEN VEHICLE', 'RESIDENTIAL ARSON', 'RESIDENTIAL BURGLARY', 'RET THEFT/DISP MERCH/<$300/PREV CONV', 'RET THEFT/DISP MERCH/>$300', 'RETAIL THEFT/DISP MERCH/<$150/2+', 'RETAIL THEFT/EMERGENCY EXIT/>$300', 'RETAIL THEFT/MOTOR FUEL/>$150', 'THEFT', 'THEFT <$300', 'THEFT <$300 PRIOR', 'THEFT <$300 WITH PRIOR', 'THEFT <$300/PRIOR CONVICT', 'THEFT <$300/SCHOOL/WORSHIP', 'THEFT >$10,000 <$100,000', 'THEFT >$10,000-$100,000', 'THEFT >$100,000-$500,000', 'THEFT >$100,000/SCHOOL/WORSHIP', 'THEFT >$300-$10,000', 'THEFT $300-$10,000/SCHOOL/WORSHIP', 'THEFT $300<$10,000', 'THEFT CONTROL INTENT <$500 PRIOR', 'THEFT CONTROL INTENT $500<$10,000', 'THEFT CONTROL INTENT 10K<100K', 'THEFT CONTROL OF PROPERTY <$500', 'THEFT DECEP INTENT $10,000<$100,000', 'THEFT DECEPTION INTENT 300<10,000', 'THEFT DECEPTION INTENT 500<10000', 'THEFT STOLEN <$300 PRIOR', 'THEFT STOLEN INTENT PERS <$300', 'THEFT STOLEN/>$300 <$10,000', 'THEFT/COIN OP MACHINE/2ND+', 'THEFT/CONTROL INTENT>$500K-1M', 'THEFT/CONTROL/INTENT >$1M', 'THEFT/DECEPTION > $100,000', 'THEFT/DECEPTION >$500,000', 'THEFT/DECEPTION/<$500 PRIOR', 'THEFT/DECEPTION/>$500 <10K', 'THEFT/EMER EXIT/<$300/PRECONV', 'THEFT/STOLEN/ <$500 PRIOR CONV', 'THEFT/STOLEN/>$10,000 <$100,000', 'THEFT/UNAUTHD CONTROL>$300<10000', 'THFT/SWITCH PRICE/<300/PRECONV', 'UNLAWFUL POSS CREDIT/DEBIT CARD', 'UNLAWFUL POSS/DRIVER/VEH/STOLEN', 'UNLAWFUL POSS>3VEH/PARTS/STOLEN', 'USE FORGED CREDIT/DEBIT CARD/>$300', 'VEHICLE THEFT CONSPIRACY', 'FORGE REGISTRATION, ETC', 'ID THEFT/USE ID/COMMIT FELONY', 'IDENTITY THEFT $2,000<$10,000', 'THEFT/FALSE REP/<$300/PRECONV', 'USE OF ACCT NUMBER/CODE/>$150', 'AID/ABET CONCEAL/MISREP VEHICLE', 'RETAIL THEFT/EMERGENCY EXIT/<$300', 'HOME REPAIR FRAUD/CONTRACT >$1,000', 'THFT/LES RTN PROP/<300/PRECONV', 'USE OF ACCT NUMBER/CODE/<$150', 'CRIM DAMAGE PROP>$100,000', 'RET THEFT/MOTOR FUEL/<$150/PREV CONV', 'UNLAWFUL USE ID CARD/THEFT', 'ATT RECEIVE/POSS/SELL STOLEN VEH', 'ATTEMPT AGG ARSON', 'ATTEMPT ARSON', 'ATTEMPT ARSON >$150', 'ATTEMPT BURGLARY', 'ATTEMPT BURGLARY/SCHOOL/PLACE WORSHIP', 'ATTEMPT POSS STOLEN TITLE/CERTIF/PLATE', 'ATTEMPT POSS STOLEN VEHICLE > $25,000', 'ATTEMPT RESIDENTIAL ARSON', 'ATTEMPT RESIDENTIAL BURGLARY', 'ATTEMPT THEFT >$10,000-$100,000', 'CONSP RESIDENTIAL BURGLARY', 'ATTEMPT AID/ABET CONCEAL/MISREP VEHICLE'])]
property_df['Holding Offense Cat'] = 'Property'

weapon_df = df[df['Holding Offense'].isin(['AGG POSS/11-20 STOLEN FIREARMS', 'AGG POSS/2-5 STOLEN FIREARMS', 'AGG UNLAWFUL USE OF WEAPON/VEH', 'AGG UNLAWFUL USE WEAPON/VEH/2ND', 'AGG UUW/ON PERSON', 'AGG UUW/PERS/FIR LOADED/FOID', 'AGG UUW/PERSON/PREV CONVICTION', 'AGG UUW/VEH/PREV CONVICTION', 'ARMED HABITUAL CRIMINAL', 'FIREARM/AMMO ACT-POSS WO/ID CARD', 'FIREARM/FOID INVALID/NO ELIG', 'GUNRUNNING', 'POSS EXPLOSIVE/INCENDIARY DEVICE', 'POSS FIREARM W/ DEFACED SER NO', 'POSS FIREARM/FOID NO ISSUE/NO ELIG', 'POSS FIREARM/LAND/GANG MEMBER', 'POSS OF FIREARM BY GANG MEMBER', 'POSSESSION OF STOLEN FIREARM', 'UNLAWFUL POSS HANDGUN/DEL/<21', 'UUW-FELON POSS WEAPON/BODY ARMOR', 'UUW-FELON POSS/USE FIREARM PRIOR', 'UUW-FELON POSS/USE FIREARM/PAROLE', 'UUW-FELON POSS/USE WEAPON/FIREARM', 'UUW/CARRY/POSS FIREARM/SCHOOL', 'UUW/MACHINE GUN/AUTO WEAPON/VEH', 'UUW/RIFLE <16 IN/SHOTGUN <18', 'UUW/CARRY/POSS FIREARM/2ND+', 'AGG UUW/PERSON/LOADED FIREARM/2+', 'UUW/CARRY/POSS FIREARM/1ST', 'ATT AGG POSS/31+ STOLEN FIREARMS', 'ATT FELON POSS/USE WEAP/BODY ARMOR', 'ATTEMPT AGG UUW W/ BODY ARMOR/PRIVATE', 'ATTEMPT UUW/FELON POSS/USE FIREARM/PAROLE', 'ATT POSS FIREARM/VEHICLE/GANG MEMBER', 'ATTEMPT POSSESSION OF STOLEN FIREARM'])]
weapon_df['Holding Offense Cat'] = 'Firearms/Weapon'

vehicle_df = df[df['Holding Offense'].isin(['AGG DUI', 'AGG DUI - SCHOOL BUS DRIVER', 'AGG DUI / 5TH DUI', 'AGG DUI / NO VALID INSURANCE', 'AGG DUI LIC SUSP OR REVOKED', 'AGG DUI/ 4TH /BAC 0.16+', 'AGG DUI/3', 'AGG DUI/3/BAC .16+', 'AGG DUI/3/PASS <16', 'AGG DUI/3RD/LIC SUSP OR REV', 'AGG DUI/3RD+', 'AGG DUI/4', 'AGG DUI/4TH/LIC SUSP OR REVOKED', 'AGG DUI/5/BAC 0.16>', 'AGG DUI/6+', 'AGG DUI/6+/BAC 0.16+', 'AGG DUI/DO DRIVER LICENSE', 'AGG DUI/LIC SUSP OR REVOKED', 'AGG DUI/NO VALID DL', 'AGG DUI/NO VALID INS', 'AGGRAVATED DUI/3RD+ DUI', 'DUI/3RD/BAC 0.16+', 'DUI/4TH+/LIC SUSP OR REVOKE', 'DUI/6TH', 'AGG DUI NO VALID DRIVERS LICENSE', 'AGG DUI/3RD+ DUI', 'DRIV LIC REVOKED/RECK HOMIC/2', 'DRIVE REVOKED/RECK HOMIC', 'DRIVE REVOKED/RECKL HOMIC/3', 'DRIVE W REVOKED/RECK HOMIC/4+', 'DRIVING RVK/SUSP DUI/SSS 4-9', 'DRIVING W/ REVOKE/SUSPEND LICENSE 2ND+', 'DRIVING W/ SUSPEND/REVOKE LICENSE 10-14', 'DRIVING W/ SUSPEND/REVOKE LICENSE 15+', 'DRIVING W/ SUSPEND/REVOKE LICENSE 2ND', 'DRIVING W/ SUSPEND/REVOKE LICENSE 3RD', 'DRIVING W/ SUSPEND/REVOKE LICENSE 4-9', 'REVOKED/SUSPENDED 2ND DUI'])]
vehicle_df['Holding Offense Cat'] = 'DUI/Vehicle Violation Without Death/Physical Injury'

registry_df = df[df['Holding Offense'].isin(['VIOLENT OFF/YOUTH FAIL REGIS NEW RESID/SCHOOL', 'VIOLENT OFF/YOUTH FAIL REGIS/FAIL TO REPORT', 'VIOLENT OFF/YOUTH FAIL REGISTER IN ILLINOIS', 'VIOLENT OFF/YOUTH FAIL REGIS IN ILLINOIS 2+', 'CHIL SEX OFFEN/RESIDE SAY CARE', 'CHILD SEX OFFENDER/PUBLIC PARK', 'CHILD SEX OFFENDER/RESIDE 500FT', 'FAIL REGIS/NEW REDIS/SCH/2ND+', 'FAIL TO REPORT ANNUALLY/2+', 'SEX OFF FAIL REGISTER NEW RESIDENCE', 'SEX OFF FAIL REPORT CHANGE ADDR/EMPL', 'SEX OFF FAIL REPORT CHANGE ADDRESS/EMPLOY', 'SEX OFF FAIL TO REPORT ANNUALLY', 'SEX OFF FAIL TO REPORT WEEKLY/2ND', 'SEX OFFENDER AT PUBLIC PARK', 'SEX OFFENDER GIVE FALSE INFO/2ND', 'SEX OFFENDER REGIS VIOLATION', 'SEX OFFENDER REGIS/FALSE INFO', 'SEX VIOL/DANGEROUS FAIL TO RPT', 'PHOTO/DIGITAL IMAGE OF CHILD BY SEX OFFENDER', 'ATT SEX OFF FAIL REPORT/NO ADDR/2', 'ATT VIOL SEX OFFENDER REGIS ACT'])]
registry_df['Holding Offense Cat'] = 'Sex Offender/MVOAY Registry Violation'

other_df = df[df['Holding Offense'].isin(['AGG FLEEING POLICE/21 MPH OVER', 'AGG FLEEING POLICE/2ND', 'AGG FLEEING/2+ CON DVC/2ND', 'AGG FLEEING/2+ DISOBEY TRAFFIC DEVICES', 'AGG FLEEING/21 MPH OVER/2ND', 'AGG FLEEING/DAMAGE >$300 PROPERTY', 'BRIBERY/OFFER BRIBE', 'BRING CANNABIS IN A PENAL INST', 'BRING CON SUBSTANCE PENAL INST', 'BRING TOOL IN A PENAL INSTITUTION', 'BRING/POSS CONTRAB IN PENAL INST', 'CAUSE/BRG CON SUB PENAL INST', 'CONCEAL HOMICIDAL DEATH', 'CONCEAL/AID FUGITIVE', 'CONSP AGAINST CIVIL RIGHTS', 'DISARM PC OFF/CORR INST EMP', 'DISARMING A PEACE OFFICER', 'ESCAPE FROM DEPT OF CORRECTION', 'ESCAPE OF FELON FROM PENAL INST', 'ESCAPE/VIOLATE ELEC MONITORING', 'FAIL REPORT ACCIDENT/DEATH/INJURY', 'FAILURE REPORT ACCIDENT/DEATH', 'FAILURE REPORT ACCIDENT/INJURY', 'FALSE ALARM/COMPLAINT TO 911', 'FALSE PERSONATION OF POLICE', 'FALSE REPORT OF OFFENSE', 'FELON ESCAPE/PEACE OFFICER', 'FELON FAIL TO RETURN FROM FURLOUGH', 'FELON PROBATIONER ESCAPE OFF', 'FLEEING/ATTEMPT ELUDE OFF 3+', 'INDIRECT CRIMINAL CONTEMPT', 'MANUF/POSS ADULTERANTS DRUG TEST', 'MONEY LAUNDERING >$500,000', 'OBSTRUCT JUST/DESTROY EVIDENCE', 'OBSTRUCTING JUSTICE', 'POSS CONTROL SUBST PENAL INST', 'POSS CONTROL SUBSTANCE IN PENAL INST', 'POSS FRAUDULENT ID CARD', 'POSS/FRAUD/DL/PERMIT', 'POSSESS CANNIBIS PENAL INST', 'POSSESS ELEC CONTRABAND PENAL INST', 'POSSESS WEAPON IN PENAL INST', 'RESIST/OBSTRUC OFFICER/INJURY', 'RICO ACQUIRE ENTERPRISE', 'RICO CONSPIRACY TO VIOLATE', 'TERRORISM/FALSE THREAT', 'THREATEN PUBLIC OFFICIAL', 'THREATEN PUBLIC OFFICIAL/2ND', 'UUW-FELON POSS WEAPON IN PRISON', 'VENDOR FRAUD/KICKBACKS VIOL', 'CRIM DAMAGE/GOVT PROP/EXPL/<$500', 'CRIM DMG/GOVT PROP/<$500', 'CRIM DMG/GOVT PROP/>$500-$10K', 'RESIST/OBSTRUCT OFFICER/INJURY', 'UUW-FELON POSS/USE WEAP/PRISON', 'CRIM DAMAGE/STATE PROP/FIRE>$100,000', 'POSS/DISP ID CARD TO ANOTHER', 'ATT FOIL/DEFEAT SCREEN TEST', 'ATTEMPT ESCAPE FELON FROM PENAL INST', 'AGGRAVATED CRUELTY TO ANIMALS', 'ANIMAL TORTURE/2ND', 'CRUELTY TO ANIMALS/2ND', 'DOG FIGHTING/ATTEND SHOW'])]
other_df['Holding Offense Cat'] = 'State/Other'

cat_offenses_df = pd.concat([injury_death_df, injury_death_unclear_df, sex_df, drugs_df, property_df, weapon_df, vehicle_df, registry_df, other_df])

# CATEGORIZE UNLISTED OFFENSES
# based on "like" criteria
# ----- these won't be 100% accurately categorized, but it should be close -----
offense_cat_dict = {'SEX OFF': 'Sex Offender/MVOAY Registry Violation',
                    'SEX': 'Sex',
                    'INDEC': 'Sex',
                    'MURDER': 'Person - Death/Physical Injury',
                    'HOMIC': 'Person - Death/Physical Injury',
                    'HARM': 'Person - Death/Physical Injury',
                    'PORN': 'Sex',
                    'DUI': 'DUI/Vehicle Violation Without Death/Physical Injury',
                    'ASSAULT': 'Person - Death/Physical Injury Unclear',
                    'BATTERY': 'Person - Death/Physical Injury Unclear',
                    'DISCH': 'Firearms/Weapon',
                    'FIREA': 'Firearms/Weapon',
                    'THEFT': 'Property',
                    'STOLE': 'Property',
                    'METH': 'Drugs',
                    'KINAP': 'Person - Death/Physical Injury Unclear',
                    'ROBB': 'Property',
                    'WEAP': 'Firearms/Weapon',
                    'GUN': 'Firearms/Weapon',
                    'POSS': 'Drugs',   # this one especially contains non-drug offenses
                    'ARSON': 'Property',
                    'ABD': 'Person - Death/Physical Injury Unclear',
                    'DEL': 'Drugs',
                    'MANS': 'Person - Death/Physical Injury',
                    'MAN': 'Drugs',
                    'ENDANGER': 'Person - Death/Physical Injury Unclear',
                    'CANNA': 'Drugs',
                    'PROP': 'Property',
                    'DRUG': 'Drugs'
                    }

# get the offenses that are uncategorized
uncat_df = df[~df.index.isin(cat_offenses_df.index)]
len2 = len(uncat_df)

# for each key word
cat_uncat_df = pd.DataFrame()
for key, val in offense_cat_dict.items():
    
    # get the offenses that contain the key word and label them with the corresponding offense type
    offense_df = uncat_df[uncat_df['Holding Offense'].str.contains(key)]
    offense_df['Holding Offense Cat'] = val
    
    # combine with the previously categorized unlisted offenses
    cat_uncat_df = pd.concat([cat_uncat_df, offense_df])
    
    # remove categorized offenses from the df of uncategorized offenses
    uncat_df = uncat_df[~uncat_df.index.isin(cat_uncat_df.index)]

# label those that are still uncategorized as other
uncat_df['Holding Offense Cat'] = 'State/Other'

# combine categorized uncategorized with uncategorized
uncat_df = pd.concat([uncat_df, cat_uncat_df])

if len(uncat_df) != len2:
    print('ERROR: categorizing uncategorized offenses changed the df length')
    1/0

# combine previously uncategorized with categorized
df = pd.concat([cat_offenses_df, uncat_df])

if len(df) != len1:
    print('ERROR: categorizing offenses changed the df length')
    1/0

df6 = df.copy()


In [15]:
# # # CLEAN INSTITUTION NAMES # # #

len1 = len(df)

# remove CC from institution names
df['last two'] = df['Parent Institution'].apply(lambda x: x[-2:])
cc_df = df[df['last two'] == 'CC']
cc_df['Parent Institution'] = cc_df['Parent Institution'].apply(lambda x: x[:-3])
non_cc_df = df[df['last two'] != 'CC']
df = pd.concat([cc_df, non_cc_df])
df.drop('last two', axis=1, inplace=True)
if len(df) != len1:
    print('cleaning institution names changed the length')
    1/0

# make institution names consistent
swil_df = df[df['Parent Institution'] == 'Southwestern IL']
swil_df['Parent Institution'] = 'Southwestern Illinois'
non_swil_df = df[df['Parent Institution'] != 'Southwestern IL']
df = pd.concat([swil_df, non_swil_df])
if len(df) != len1:
    print('more cleaning institution names changed the length')
    1/0

df7 = df.copy()


In [16]:
# # # CALCULATE AGES # # #

# ----- easier to do this in tableau for now; debug when there's more time -----

# # calculate ages

# len1 = len(df)

# def calc_age(cur_df, col, age_type):
#     len2 = len(cur_df)

#     non_blank_col_df = cur_df[~pd.isnull(cur_df[col])]
#     non_blank_col_df[age_type] = np.floor((non_blank_col_df[col] - non_blank_col_df['Date of Birth']).dt.days() / 365.25)
#     blank_col_df = cur_df[pd.isnull(cur_df[col])]
#     cur_df = pd.concat([non_blank_col_df, blank_col_df])
    
#     if len(cur_df) != len2:
#         print('ERROR: ' + age_type + 'changed the df length')
#         1/0
    
#     return cur_df

# df = calc_age(df, 'Doc Date', 'Age')

# df = calc_age(df, 'Projected Mandatory Supervised Release (MSR) Date', 'Age at Release')

# df = calc_age(df, 'Custody Date', 'Estimated Age at Custody')

# df = calc_age(df, 'Projected Discharge Date', 'Estimated Age at Discharge')


# # est age at release = age at release where available, age at discharge where not
# non_blank_msr_df = df[~pd.isnull(df['Projected Mandatory Supervised Release (MSR) Date'])]
# non_blank_msr_df['Estimated Age at Release'] = non_blank_msr_df['Age at Release']
# pos_non_blank_msr_df = non_blank_msr_df[non_blank_msr_df['Age at Release'] > 0]

# other_msr_df = df[~df.index.isin(pos_non_blank_msr_df.index)]
# non_blank_msr_df['Estimated Age at Release'] = non_blank_msr_df['Estimated Age at Discharge']

# df = pd.concat([non_blank_msr_df, pos_non_blank_msr_df])

# if len(df) != len1:
#     print('ERROR: calculating ages changed the df length')
#     1/0


In [17]:
# # # ADD ADMISSION TYPE CATEGORIES # # #

len1 = len(df)

non_blank_admission_df = df[~pd.isnull(df['Admission Type'])]
blank_admission_df = df[pd.isnull(df['Admission Type'])]
blank_admission_df['Admission Type'] = 'UNKNOWN'
df = pd.concat([non_blank_admission_df, blank_admission_df])

df['Admission Type'] = df['Admission Type'].apply(lambda x: x.upper().strip())
df['Admission Type'] = df['Admission Type'].replace('CONDITIONAL RELEASE, NEW SENT', 'CONDITIONAL RELEASE, NEW SENTENCE').replace('WORK RELEASE VIOLATOR, NEW SENTNCE', 'WORK RELEASE VIOLATOR, NEW SENTENCE')

admission1_df = df[df['Admission Type'].isin(['DIRECT FROM COURT', 'TRANSFERRED FROM JUVENILE', 'JUV PAROLE TO ADULT PAROLE'])]
admission1_df['Admission Type--Recidivism'] = 'First IDOC Sentence'

admission2_df = df[df['Admission Type'].isin(['DISCHARGED & RECOMMITTED', 'MSR VIOLATOR, NEW SENTENCE', 'PAROLE VIOLATOR, NEW SENTENCE', 'WORK RELEASE VIOLATOR, NEW SENTENCE', 'CONDITIONAL RELEASE, NEW SENTENCE'])]
admission2_df['Admission Type--Recidivism'] = 'Second/+ IDOC Sentence'

parole_df = df[df['Admission Type'].isin(['TECHNICAL PAROLE VIOLATOR', 'TECHNICAL MSR VIOLATOR', 'CONDITIONAL RELEASE VIOLATOR', 'TECHNICAL VIOLATORS', 'WORK RELEASE VIOLATOR'])]
parole_df['Admission Type--Recidivism'] = 'Technical MSR/Parole Violator'
               
unknown_df = df[df['Admission Type'].isin(['COVID-19 COUNTY ADMIT', 'RETURN TO CUSTODY', 'RETURN ADDITIONAL MITTIMUS', 'ADMIT FROM OTHER CUSTODY', 'NOT IN IDOC CUSTODY', 'COOK COUNTY ADMISSION', 'GRAHAM CO JAIL ADM', 'LOGAN CO JAIL ADM', 'MENARD CO JAIL ADM', 'NORTHERN CO JAIL ADM', 'UNKNOWN', 'NONE', 'OTHER'])]
unknown_df['Admission Type--Recidivism'] = 'Unknown'
                
cat_admission_df = pd.concat([admission1_df, admission2_df, parole_df, unknown_df])

uncat_df = df[~df.index.isin(cat_admission_df.index)]
print('\nUncategorized')
print(uncat_df['Admission Type'].unique())
uncat_df['Admission Type--Recidivism'] = 'Uncategorized'

df = pd.concat([cat_admission_df, uncat_df])

if len(df) != len1:
    print('ERROR: categorizing admission types changed the length')
    1/0

df8 = df.copy()



Uncategorized
['EDV' 'COURT ADMISSIONS' 'NEW SENTENCE VIOLATORS']


In [18]:
# # # CALCULATE ELIGIBILITY FOR SENTENCING CREDITS # # #

len1 = len(df)

# apply the calculation
non_eligible_df = df[((df['TIS Pct'] >= 0.85) & (~pd.isnull(df['TIS Pct'])))
                    | df['Holding Offense'].str.contains('GUNRUNNING')
                    | df['Sentence Years'].isin(['LIFE', 'SDP'])
                    | (df['Holding Offense Cat'] == 'Sex')]
non_eligible_df['EEC'] = 'NO'

eligible_df = df[~df.index.isin(non_eligible_df.index)]
eligible_df['EEC'] = 'YES'

df = pd.concat([non_eligible_df, eligible_df])

df9 = df.copy()


In [19]:
# # # EXPORT # # #

# export full data set
df.to_csv('df.csv')

# # export full data set for most recent doc date
# df_recent = df[df['Doc Date'] == date(2021, 6, 30)]

# # export single institution all time
# df_export = df[df['Parent Institution'] == 'Logan']
# df_export.to_csv('logan_df.csv')


In [20]:
short_admis_date_diff_df.to_csv('short_admis_date_diff_df.csv')